In [ ]:
import polars as pl
import pandas as pd
import requests

In [ ]:
def add_metro_and_dist(metro_df, df):

    lat_array = np.array(metro_df['lat'])
    lon_array = np.array(metro_df['lng'])
    station_dct = {index: station for index, station in enumerate(metro_df['station'])}
    coefficient = 40000 / 360

    def compute_closest(lat, lon):
        distances = (lat_array - lat) ** 2 + (lon_array - lon) ** 2
        index = np.argmin(distances)
        return index, distances[index]
    
    closest_metro = []
    distances = []
    
    for lat, lon in zip(df['lat'], df['lng']):
        index, value = compute_closest(lat, lon)
        closest_metro.append(station_dct[index])
        distances.append(np.sqrt(value) * coefficient)
        
    df['closest_station'] = closest_metro
    df['dist_to_station'] = distances
    
    return df

## Msk

In [ ]:
msk_data_prepared = pd.read_csv(r"C:\Users\sahab\Downloads\Telegram Desktop\data.csv")
msk_data_prepared.head(5)

In [ ]:
stations = pd.read_csv(r"C:\Users\sahab\Downloads\Telegram Desktop\stations.csv")
stations.head(5)

In [ ]:
headers = {
    'Accept': 'application/json',
}

section_json = requests.get('https://inpars.ru/api/v2/estate/section?access-token=aEcS9UfAagInparSiv23aoa_vPzxqWvm', headers=headers).json()

section_map = {
    i["id"]: i["title"] for i in section_json["data"]
}

msk_data_prepared["section_name"] = msk_data_prepared.sectionId.apply(lambda x: section_map[x])


In [ ]:
category_json = requests.get('https://inpars.ru/api/v2/estate/category?access-token=aEcS9UfAagInparSiv23aoa_vPzxqWvm', headers=headers).json()

category_map = {
    i["id"]: i["title"] for i in category_json["data"]
}

msk_data_prepared["category_name"] = msk_data_prepared.categoryId.apply(lambda x: category_map[x])

In [ ]:
msk_data_prepared = add_metro_and_dist(stations, data_spb_raw)

msk_data_prepared = msk_data_prepared.drop(columns=["closest_metro", "sectionId", 'categoryId', 'updated'])

## Spb

In [ ]:
spb_data_prepared = pd.read_csv(r"C:\Users\sahab\spb.csv")
spb_data_prepared.head(5)

In [ ]:
spb_data_prepared["section_name"] = spb_data_prepared.sectionId.apply(lambda x: section_map[x])

spb_data_prepared["category_name"] = spb_data_prepared.categoryId.apply(lambda x: category_map[x])

data_spb_prepared = add_metro_and_dist(stations, data_spb_raw)

data_spb_prepared = data_spb_prepared.drop(columns=["closest_metro", "sectionId", 'categoryId', 'updated'])